### Propositions pour l'onglet ACTION.

#### Objectifs

Objectif de l'onglet ACTION : 
- Présentation des actions passées avec des KPI (KPI à définir).
- Présentation des actions à venir

#### Contexte 

Besoins des collectivités :​
-  Mieux connaître la problématique macrodéchets sur leur territoire​
-  Suivre l'efficacité des politiques publiques mises en œuvre​

Indicateurs déjà identifiés par Merterre : 
- Typologie des matériaux​
- Top déchets comptés,​
- Typologie des déchets (si niveau 4)​
- Secteurs économiques concernés, filières REP concernés, ​
- Filtres par milieu/lieu pur chacune de ces informations​
- Evolutions dans le temps

Analyses présentes dans le bilan pdf : 
- Typologie des matériaux 
    - Pie chart : part des matériaux en volume tous milieux confondus 
    - Stacked bar chart : part des matériaux en volume selon le type de milieu
    - Sorted bar chart : Principaux déchets comptés (en nombre d'objets)
- Typologie des déchets 
    - pie chart fragments vs objets identifiables
    - pie chart en nombre du matériau pour les fragments ramassés
    - part en nombre des objets identifiables ramassés
- Secteurs économiques : 
    - pie chart : part des déchets dont le secteur est identifiable
    - pie chart des secteurs économiques identifiés, tous milieux confondus
    - graph à barre des secteurs économiques identifiés en fonction du type de milieu

### Import

In [56]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

import seaborn as sns
import matplotlib.pyplot as plt
import folium
import streamlit as st
from streamlit_folium import folium_static
from streamlit_folium import st_folium

from IPython.display import display, HTML

import locale

### Exploration

In [59]:
PATH = "./data/"

# nombre maximum de colonnes à afficher = 1500
pd.set_option('display.max_columns', 1500)

# import données structures et niveaux de caracterisation et fichier geojson france
#df_niveaux_dechets = pd.read_excel(PATH+"Déchets-Niveaux2-3.xlsx")
#df_structures = pd.read_excel(PATH+"export_structures_29022024.xlsx")
#gdf = gpd.read_file(PATH+"a-com2022.json")

# import du fichier excel des données nettoyées par le sript python "cleaning.py" dans un dataframe et visualisation des premieres lignes
#df_enriched_bdd_zds = pd.read_excel(PATH + "data_zds_enriched.xlsx")

# import du fichier d'exportations des events 
df_events_raw = pd.read_excel(PATH + "export_events_14032024.xlsx")

In [3]:
df_events_raw.head()

,ID_EVENEMENT,NOM_EVENEMENT,TYPE_EVENEMENT,EVENEMENT_ENVERGURE,NOM_ZONE,NOM_SPOT_A1S,DATE,TYPES_MILIEUX,REGION,DEPARTEMENT,NIVEAU_CARAC,NOM_VILLE,CODE_POSTAL,COORD_GPS_RDV,ANNULE,NOM_STRUCTURE,ID_STRUCTURE,REFERENT_STRUCTURE,TELEPHONE_STRUCTURE,COURRIEL_STRUCTURE
0,3464,Cleanwalk à Toulouse,ramassage,NaN,NaN,NaN,2024/04/27,"Zone urbaine/Route, rue, place/Présent au sol ...",Occitanie,Haute-Garonne,2.0,Toulouse,31000,"[1.4358240363526,43.610577467027]",0,Cleanwalker Toulouse,748,Laureen Aguiriano,0760171045,cleanwalker.toulouse@gmail.com
1,3463,Ramassage collège Sacré Coeur Aix-en-Provence,ramassage,NaN,NaN,NaN,2024/03/12,Zone urbaine/Parc urbain - Jardin public/Prése...,Région Sud Provence Alpes Côtes D'Azur,Bouches-du-Rhône,2.0,Aix-en-Provence,13626,"[5.4568544625909,43.531501471356]",0,Le Naturoscope,10,NaN,0491755843,contact@naturoscope.fr
2,3462,Nettoyons le Sud à l'Escale Borely,ramassage,Nettoyons le Sud,NaN,NaN,2024/04/13,"Littoral (terrestre)/Plage (sable, galets, gra...",Région Sud Provence Alpes Côtes D'Azur,Bouches-du-Rhône,2.0,Marseille,13008,"[5.3752756099129,43.25501780853]",0,1 déchet par jour,88,NaN,0624667835,jeramasse@1dechetparjour.com
3,3461,Ramassage à l'Anse de Malmousque,ramassage,NaN,NaN,NaN,2024/03/23,Littoral (terrestre)/Digue et ouvrage/Présent ...,Région Sud Provence Alpes Côtes D'Azur,Bouches-du-Rhône,2.0,Marseille,13007,"[5.3478205199099,43.28060304739]",0,1 déchet par jour,88,NaN,0624667835,jeramasse@1dechetparjour.com
4,3460,Ramassage sur l'ATE,ramassage,NaN,NaN,NaN,2024/02/29,Zone naturelle ou rurale (hors littoral et mon...,Occitanie,Hérault,2.0,Agde,34300,"[3.5163035988808,43.297570894043]",0,Ecole Notre-Dame,744,Yves DANDOY,+33675624930,dandoy.y@gmail.com


In [4]:
df_events_raw.shape

(2960, 20)

In [5]:
df_events_raw.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2960 entries, 0 to 2959
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID_EVENEMENT         2960 non-null   int64  
 1   NOM_EVENEMENT        2960 non-null   object 
 2   TYPE_EVENEMENT       2960 non-null   object 
 3   EVENEMENT_ENVERGURE  1252 non-null   object 
 4   NOM_ZONE             945 non-null    object 
 5   NOM_SPOT_A1S         640 non-null    object 
 6   DATE                 2960 non-null   object 
 7   TYPES_MILIEUX        2858 non-null   object 
 8   REGION               2930 non-null   object 
 9   DEPARTEMENT          2857 non-null   object 
 10  NIVEAU_CARAC         2873 non-null   float64
 11  NOM_VILLE            2943 non-null   object 
 12  CODE_POSTAL          2894 non-null   object 
 13  COORD_GPS_RDV        2959 non-null   object 
 14  ANNULE               2960 non-null   int64  
 15  NOM_STRUCTURE        2960 non-null   o

In [6]:
list(df_events_raw.columns)

['ID_EVENEMENT',
 'NOM_EVENEMENT',
 'TYPE_EVENEMENT',
 'EVENEMENT_ENVERGURE',
 'NOM_ZONE',
 'NOM_SPOT_A1S',
 'DATE',
 'TYPES_MILIEUX',
 'REGION',
 'DEPARTEMENT',
 'NIVEAU_CARAC',
 'NOM_VILLE',
 'CODE_POSTAL',
 'COORD_GPS_RDV',
 'ANNULE',
 'NOM_STRUCTURE',
 'ID_STRUCTURE',
 'REFERENT_STRUCTURE',
 'TELEPHONE_STRUCTURE',
 'COURRIEL_STRUCTURE']

## Nettoyage

In [ ]:
# Copie avant de faire des modifs
df_events = df_events_raw.copy()

# Supprime les événements annulés
df_events = df_events[df_events.ANNULE != 1]

# Convertit le type de la colonne 'DATE' en datetime
df_events.DATE = pd.to_datetime(df_events.DATE, yearfirst=True)

# Remplace les valeurs NaN par 0 dans le NIVEAU DE CARAC
df_events.NIVEAU_CARAC.fillna(0, inplace=True)
# change le type de NIVEAU_CARAC en int
df_events.NIVEAU_CARAC = df_events.NIVEAU_CARAC.astype('int64')

# COORD GPS : 
# Supprime les crochets 
# split les valeurs par la virgule pour obtenir les coordonnées X et Y 
# convertit en float
df_events[['COORD_GPS_X', 'COORD_GPS_Y']] = df_events.COORD_GPS_RDV.str.strip('[]').str.split(',', expand=True).astype(float)

# Majuscule sur TYPE_EVENEMENT
df_events.TYPE_EVENEMENT = df_events.TYPE_EVENEMENT.str.capitalize()

# Supprime les colonnes de contact des structures (anonymisation)
# Supprime la colonne COORD_GPS_RDV (inutile)
# Supprime la colonne ANNULE (déjà filtrée)
df_events = df_events.drop(['REFERENT_STRUCTURE', 'TELEPHONE_STRUCTURE', 'COURRIEL_STRUCTURE', 'COORD_GPS_RDV', 'ANNULE'], axis=1)

In [8]:
df_events.to_csv(PATH + 'export_events_cleaned.csv', index=False)

## Viz

In [ ]:
# Filtrer les événements à venir
df_events_a_venir = df_events[df_events.DATE > (datetime.now() - timedelta(days=5))]

# Trie les events par date 
df_events_a_venir.sort_values(by='DATE', inplace=True)

# Coord approximatives du centre de la France
coord_centre_france = [46.603354, 1.888334]

In [ ]:
# Resize la cellule contenant la map
cellMap = folium.Figure(width=1000, height=1000)
# Crée map centrée sur une position initiale
carte = folium.Map(location=coord_centre_france, zoom_start=6).add_to(cellMap)

# Ajout des marqueurs pour chaque événement à venir
for idx, row in df_events_a_venir.iterrows():
    folium.Marker(location=[row.COORD_GPS_Y, row.COORD_GPS_X], popup=row.NOM_EVENEMENT).add_to(carte)

# Affiche la carte
carte

In [63]:
# Resize la cellule contenant la map
cellMap = folium.Figure(width=800, height=800)

# Crée la carte centrée sur une position initiale
carte = folium.Map(location=coord_centre_france, zoom_start=6).add_to(cellMap)

# Ajout des marqueurs pour chaque événement à venir
noms_evenements = []
for idx, row in df_events_a_venir.iterrows():
    nom_evenement = row.NOM_EVENEMENT
    nom_structure = row.NOM_STRUCTURE
    type_evenement = row.TYPE_EVENEMENT
    jour_numero = str.format("%02d" % (row.DATE.day))
    mois_court = row.DATE.strftime("%b")
    annee = row.DATE.year
    noms_evenements.append((nom_evenement, nom_structure, type_evenement, jour_numero, mois_court))
    folium.Marker(location=[row.COORD_GPS_Y, row.COORD_GPS_X], popup=nom_evenement, tooltip=nom_evenement).add_to(carte)

table_html = '<div><h2 style="font-weight:bold;">Événements à venir</h2>'
table_html += '<table>'
for nom_evenement, nom_structure, type_evenement, jour_numero, mois_court in noms_evenements:
    table_html += f'<tr><td style="vertical-align: top;"><div>{jour_numero}</div><div>{mois_court}</div></td>'
    table_html += f'<td style="margin-left: 20px; text-align: left;"><span style="font-weight:bold;">{nom_evenement}</span><br>'
    table_html += f'<span style="text-align: left;">{type_evenement}</span> | <span>{nom_structure}</span></td></tr>'
table_html += "</table></div>"

# JavaScript pour mettre à jour les informations lorsqu'un marqueur est cliqué
#javascript = """
#<script>
#function updateEventInfo(event) {
#    var eventName = event.target._popup._content;
#    document.getElementById('event_info').innerText = eventName;
#}
#var markers = document.getElementsByClassName('leaflet-marker-icon');
#for (var i = 0; i < markers.length; i++) {
#    markers[i].addEventListener('click', updateEventInfo);
#}
#</script>
#"""


# Afficher la carte et la table dans un cadre d'affichage HTML avec JavaScript
display(HTML(f'<div style="display:flex;"><div style="flex:1;">{cellMap._repr_html_()}</div><div style="flex:1;">{table_html}</div></div>'))

30mars,Nettoyons l’Eygoutier avant que les déchets n’arrivent à la mer !Ramassage | Chercheurs en herbe
13avr.,Nettoyons le Sud à l'Escale BorelyRamassage | 1 déchet par jour
13avr.,"Collecte de déchets sur la plage du Mus de Loup ( La tremblade 17 ) samedi 13 avril 2024Ramassage | MALINE (Mouvement d'Actions pour le Littoral, la Nature et l'Environnement)"
13avr.,Nettoyons le SudRamassage | Planète Perles
13avr.,Nettoyons le SudRamassage | Planète Perles
13avr.,On nettoie le Sud à Borely !!Ramassage | Mer Veille
20avr.,Collecte de Déchets Sauvages Almanarre / MadragueRamassage | Explore & Preserve
27avr.,Cleanwalk à ToulouseRamassage | Cleanwalker Toulouse
06mai,Nettoyage de la plage du CouchantRamassage | LABELBLEU
01juin,Calanques Propres 2024 ! Ramassage aquatique sur le secteur Maronaise - Les GoudesRamassage | Mer Veille
07juil.,Coup de propre sur le Vallon des Auffes !Ramassage | Mer Veille


In [16]:
# Créer une carte centrée sur une position initiale
carte = folium.Map(location=coord_centre_france, zoom_start=6)

# Ajouter des marqueurs pour chaque événement à venir
for idx, row in df_events_a_venir.iterrows():
    folium.Marker(location=[row.COORD_GPS_Y, row.COORD_GPS_X], popup=row.NOM_EVENEMENT).add_to(carte)

# Afficher la carte avec Streamlit
st.write("Carte des événements à venir")
st_data = st_folium(carte)
#folium_static(carte)


2024-03-28 16:32:39.101 
  command:

    streamlit run c:\Users\msaid\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
